# Описание проекта - Бобкова Ксения

В данной тетрадке кратко описано выполнение поставленной задачи. Файлы с кодом и полученной базой данных также есть в Гитхабе.

Написание кода состояло из двух основных этапов: создание базы данных (включая предобработку текстов) и написание функции поиска.

### Создание базы данных

In [ ]:
import sqlite3

from bs4 import BeautifulSoup
from pymystem3 import Mystem
import nltk
import os
import glob

#Создание таблиц для базы данных
def get_db_connect(db_name):
  conn = sqlite3.connect(db_name)
  cur = conn.cursor()

  cur.execute("CREATE TABLE IF NOT EXISTS text"
              " (text_id INTEGER PRIMARY KEY AUTOINCREMENT, text_info TEXT UNIQUE);")

  cur.execute("CREATE TABLE IF NOT EXISTS sentence"
              " (sent_id INTEGER PRIMARY KEY AUTOINCREMENT, text_id INTEGER, sent_text TEXT);")

  cur.execute("CREATE TABLE IF NOT EXISTS onegramm"
              " (sent_id INTEGER, word_index INTEGER, word_id INTEGER, word_ind, word_form TEXT"
              ", CONSTRAINT primarykey_sent_word PRIMARY KEY (sent_id,word_index,word_id));")

  cur.execute("CREATE TABLE IF NOT EXISTS bigramm"
              " (sent_id INTEGER, word_index INTEGER, word1_id INTEGER, word1_form TEXT, word2_id INTEGER, word2_form TEXT"
              ", CONSTRAINT primarykey_sent_word1_word2 PRIMARY KEY (sent_id,word_index,word1_id,word2_id));")

  cur.execute("CREATE TABLE IF NOT EXISTS trigramm"
              " (sent_id INTEGER, word_index INTEGER, word1_id INTEGER, word1_form TEXT, word2_id INTEGER, word2_form TEXT, word3_id INTEGER, word3_form TEXT"
              ", CONSTRAINT primarykey_sent_word1_word2_word3 PRIMARY KEY (sent_id,word_index,word1_id,word2_id,word3_id));")

  cur.execute("CREATE TABLE IF NOT EXISTS word"
              " (word_id INTEGER PRIMARY KEY AUTOINCREMENT, lemma TEXT, pos_tag TEXT"
              ", CONSTRAINT lemma_pos_tag UNIQUE (lemma,pos_tag));")

  conn.commit()

  return conn

#Обработка файлов с текстами и запись полученных из них данных в таблицы (в цикле по папке)
#1) Получение из файла сведений о нем (в переменную text_info), очистка текста файла (в переменную cleaned_dd_text)
#2) Разбивка текста на предложения (sentence_list)
#3) Использование Mystem для получения части речи, леммы, словоформы
#4) Кроме того, сразу же были сформированы биграммы и триграммы для записи в бд

def db_create(db_conn: sqlite3.Connection):
  dir_path = os.getcwd() + "/texts000"

  mystem = Mystem()

  for filename in glob.glob(os.path.join(dir_path, "*.shtml")):
    with open(os.path.join(dir_path, filename), 'r', encoding="windows-1251") as fhtml:
      print("processing {filik}".format(filik=filename))
      db_cur = db_conn.cursor()

      contents = fhtml.read()
      soup = BeautifulSoup(contents, 'html.parser')

      text_info = soup.find("title").string
    
      db_cur.execute(
        "INSERT INTO text (text_info) VALUES(?) ON CONFLICT(text_info) DO NOTHING RETURNING text_id;", (text_info, ))
      text_id = db_cur.fetchone()
      if text_id is None:
        text_id = db_cur.execute("SELECT text_id FROM text WHERE text_info=?", (text_info, )).fetchone()
      text_id = text_id[0]

      all_dd_text = soup.dd.text

      cleaned_dd_text = all_dd_text.replace("\n", '')
      cleaned_dd_text = cleaned_dd_text.replace("\xa0", '')

      sentence_list = nltk.sent_tokenize(cleaned_dd_text)

      for sentence in sentence_list:
        sent_text = sentence
        db_cur.execute("INSERT INTO sentence (text_id,sent_text) VALUES(?,?) RETURNING sent_id;", (text_id, sent_text))
        sent_id = db_cur.fetchone()[0]

        ma_list = mystem.analyze(sentence)
        word_index = 0
        prev_words = [None, None]
        for w_analyse in ma_list:
          word_form = w_analyse.get('text', '').lower()
          analyze_list = w_analyse.get('analysis', [])
          if len(analyze_list) > 1:
            pass

          if len(analyze_list) > 0:
            for analyze_dict in analyze_list:
              lemma = analyze_dict.get('lex', 'EMPTY')
              gr = analyze_dict.get('gr', '')
              pos_tag = (gr.split("=")[0].split(",")[0]) if len(gr) > 0 else ''
            
#Для наглядности прогресса обработки данных 
              print(text_id, sent_id, sent_text, word_form, lemma, pos_tag)

              db_cur.execute("INSERT INTO word (lemma,pos_tag) VALUES(?,?) ON CONFLICT(lemma,pos_tag) DO NOTHING RETURNING word_id;", (lemma, pos_tag))
              word_id = db_cur.fetchone()
              if word_id is None:
                word_id = db_cur.execute("SELECT word_id FROM word WHERE lemma=? AND pos_tag = ?", (lemma, pos_tag)).fetchone()
              word_id = word_id[0]

              db_cur.execute("INSERT OR IGNORE INTO onegramm (sent_id,word_index,word_id,word_form) VALUES(?,?,?,?);", (sent_id, word_index, word_id, word_form))

              if word_index > 0:
                word1_id, word1_form = prev_words[1]
                db_cur.execute("INSERT OR IGNORE INTO bigramm (sent_id,word_index,word1_id,word1_form,word2_id,word2_form)"
                               " VALUES(?,?,?,?,?,?);", (sent_id, word_index-1, word1_id, word1_form, word_id, word_form))

              if word_index > 1:
                word1_id, word1_form = prev_words[0]
                word2_id, word2_form = prev_words[1]
                db_cur.execute("INSERT OR IGNORE INTO trigramm (sent_id,word_index,word1_id,word1_form,word2_id,word2_form,word3_id,word3_form)"
                               " VALUES(?,?,?,?,?,?,?,?);", (sent_id, word_index-2, word1_id, word1_form, word2_id, word2_form, word_id, word_form))

#Для построения биграмм и триграмм сохранялись предыдущие слова относительно анализируемого
            word_index += 1
            prev_words.append(tuple([word_id, word_form]))
            prev_words.pop(0)

  db_conn.commit()


def db_main():
  db_name = "f_corpus.sqlite"
  db_conn = get_db_connect(db_name)
  db_create(db_conn)

  db_conn.close()


db_main()


### Функция поиска

In [4]:
! pip install pymystem3

  Using cached pymystem3-0.2.0-py3-none-any.whl (10 kB)


In [ ]:
import sqlite3
from pymystem3 import Mystem


def get_db_read_connect(db_name):
  conn = sqlite3.connect(db_name)
  return conn

def get_word_condition(num: int, word: str):
    
  mystem=Mystem()

#Если требовалось найти определенную словоформу

  sql_where = '' 
    
  if word.startswith('"'):  # word form condition
    word_form = word[1:-1]
    word_form = word_form.lower()
    sql_where = "form{num}='{frm}'".format(num=num, frm=word_form)
    
  else:#Если требовалось найти по лемме
    lemma = mystem.lemmatize(word)[0]
    
    sql_where = "lem{num}='{lem}'".format(num=num, lem=lemma)

  return sql_where


def db_search_main():
  pos_tag_set = {"A", "ADV", "ADVPRO", "ANUM", "APRO", "COM", "CONJ", "INTJ", "NUM", "PART", "PR", "S", "SPRO", "V"}

  print('Для ввода словоформы напишите слово в кавычках (например, "смешной")')
  print('Для ввода леммы напишите слово без кавычек (например, смешной)')
  print('Для поиска части речи, пожалуйста, введите тэг (A, ADV, ADVPRO, ANUM, APRO, COM, CONJ, INTJ, NUM, PART, PR, S, SPRO, V) (например, V)')
  print('Для поиска леммы или словоформы определнной части речи напишите между ними + (например, "гуляю"+V)')
  print('Максимальная длина запроса 3, вводите элементы через пробел')

  query = input()
  # query = 'V "мне"+SPRO S'
  # query = 'V "мне"'
  # query = 'V'

#Анализ введенного запроса, исходя из его длины
  token_list = query.split(' ')
  if len(token_list) == 1:
    sql_select = "SELECT t.text_id,t.text_info,s.sent_text" \
                 ",g1.word_form AS form1,w1.lemma AS lem1,w1.pos_tag AS pos1" \
                 " FROM text AS t" \
                 " INNER JOIN sentence AS s on s.text_id=t.text_id" \
                 " INNER JOIN onegramm AS g1 on g1.sent_id=s.sent_id" \
                 " INNER JOIN word AS w1 on w1.word_id=g1.word_id"
  elif len(token_list) == 2:
    sql_select = "SELECT t.text_id,t.text_info,s.sent_text" \
                 ",g2.word1_form AS form1,w1.lemma AS lem1,w1.pos_tag AS pos1" \
                 ",g2.word2_form AS form2,w2.lemma AS lem2,w2.pos_tag AS pos2" \
                 " FROM text AS t" \
                 " INNER JOIN sentence AS s on s.text_id=t.text_id" \
                 " INNER JOIN bigramm AS g2 on g2.sent_id=s.sent_id" \
                 " INNER JOIN word AS w1 on w1.word_id=g2.word1_id" \
                 " INNER JOIN word AS w2 on w2.word_id=g2.word2_id"
  elif len(token_list) == 3:
    sql_select = "SELECT t.text_id,t.text_info,s.sent_text" \
                 ",g3.word1_form AS form1,w1.lemma AS lem1,w1.pos_tag AS pos1" \
                 ",g3.word2_form AS form2,w2.lemma AS lem2,w2.pos_tag AS pos2" \
                 ",g3.word3_form AS form3,w3.lemma AS lem3,w3.pos_tag AS pos3" \
                 " FROM text AS t" \
                 " INNER JOIN sentence AS s on s.text_id=t.text_id" \
                 " INNER JOIN trigramm AS g3 on g3.sent_id=s.sent_id" \
                 " INNER JOIN word AS w1 on w1.word_id=g3.word1_id" \
                 " INNER JOIN word AS w2 on w2.word_id=g3.word2_id" \
                 " INNER JOIN word AS w3 on w3.word_id=g3.word3_id"
  else:
    print('Некорректный запрос, в следующий раз введите запрос длиной от 1 до 3')
    exit()

  token_num = 1
  sql_where_list = [] #Составление запроса в бд

    #Анализ каждого ввденного элемента запроса (ввели лемму/словоформу/часть речи)
  for token in token_list:
    if token in pos_tag_set:  # POS tag
      sql_where_list.append("pos{num}='{tkn}'".format(num=token_num, tkn=token))
    elif '+' in token:        # word+POS_tag
      word, pos_tag = tuple(token.split('+'))
      sql_where_list.append(get_word_condition(token_num, word))
      sql_where_list.append("pos{num}='{tkn}'".format(num=token_num, tkn=pos_tag))
    else:                     # word form
      sql_where_list.append(get_word_condition(token_num, token))

    token_num += 1

    #Отправка запроса в бд
  db_name = "f_corpus.sqlite"
  sql_query = sql_select + ' WHERE ' + " AND ".join(sql_where_list)


  db_conn = get_db_read_connect(db_name)
  db_cur = db_conn.cursor()

  db_cur.execute(sql_query)
  row_list = db_cur.fetchall()
  db_conn.close()

  if len(row_list) == 0:
    print('Увы, ничего не нашлось:(')

  row_num = 1
  for row in row_list:
    print(row_num, ') Мета-информация: ', row[1])
    print('Предложение: ', row[2])
    row_num += 1


db_search_main()


В презентации проекта более подробно будет сказано о схеме базы данных, примерах запросов, проблемах, которые возникли, предложениях по оптимзации кода.